# Client Engagement Letter Draft — Hands-on Tutorial
This notebook walks through configuring the **ClientEngagementLetterDraft** step, creating sample inputs, running the automation, and reviewing the generated outputs.

In [ ]:
import sys, subprocess
print(sys.version)

def _ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])
for _pkg in ['pandas', 'openpyxl', 'requests']:
    _ensure(_pkg)
from pathlib import Path
ROOT = Path.cwd()
sys.path.append(str(ROOT / 'src'))
print('Project source path added:', ROOT / 'src')


## 1. Simulate practice management API calls
We'll mock REST API clients for **Deltek Vantagepoint** and **Practice CS** so the tutorial mirrors how metadata would be pulled from those systems before running the automation.

In [ ]:
import json
from datetime import datetime
import requests

class MockAPISession(requests.Session):
    """A lightweight session that returns canned payloads for specific routes."""

    def __init__(self, payloads):
        super().__init__()
        self.payloads = payloads

    def get(self, url, params=None, **kwargs):
        params = params or {}
        key = (url, tuple(sorted(params.items())))
        payload = self.payloads.get(key)
        response = requests.Response()
        response.status_code = 200 if payload is not None else 404
        response._content = json.dumps(payload or {"error": "Not found"}).encode('utf-8')
        response.headers['Content-Type'] = 'application/json'
        prepared = requests.Request('GET', url, params=params).prepare()
        response.url = prepared.url
        response.request = prepared
        if response.status_code >= 400:
            print('[MockAPISession] 404 for', response.url)
        else:
            print('[MockAPISession] GET', response.url)
        return response

class MockDeltekVantagepointAPI:
    """Simulate the Engagement Management endpoints."""

    def __init__(self, session: requests.Session):
        self.session = session

    def fetch_client_metadata(self, fiscal_year: str):
        response = self.session.get(
            'https://api.deltek.mock/v1/engagements',
            params={'fiscal_year': fiscal_year},
        )
        response.raise_for_status()
        payload = response.json()
        print('Deltek payload generated on', payload['generated_on'])
        return payload['engagements']

class MockPracticeCSAPI:
    """Simulate the Practice CS client validation and service line lookups."""

    def __init__(self, session: requests.Session):
        self.session = session

    def validate_client_ids(self, client_ids):
        response = self.session.get(
            'https://api.practicecs.mock/v1/clients/validate',
            params={'client_ids': ','.join(sorted(client_ids))},
        )
        response.raise_for_status()
        payload = response.json()
        print('Validated IDs →', payload['valid_ids'])
        return payload['valid_ids']

    def fetch_service_lines(self, client_ids):
        response = self.session.get(
            'https://api.practicecs.mock/v1/service-lines',
            params={'client_ids': ','.join(sorted(client_ids))},
        )
        response.raise_for_status()
        payload = response.json()
        catalog = {}
        for item in payload['service_lines']:
            catalog[item['ServiceLineCode']] = {
                'Description': item['Description'],
                'Rate': item['Rate'],
            }
        print('Practice CS catalog →', catalog)
        return catalog

deltek_session = MockAPISession(
    {
        (
            'https://api.deltek.mock/v1/engagements',
            (('fiscal_year', '2025'),),
        ): {
            'generated_on': datetime.utcnow().isoformat(),
            'engagements': [
                {
                    'ClientID': 'C-1001',
                    'ClientName': 'Acme Holdings',
                    'FiscalYear': '2025',
                    'ServiceLines': ['CONSULT', 'TAX'],
                },
                {
                    'ClientID': 'C-2040',
                    'ClientName': 'Global Manufacturing',
                    'FiscalYear': '2025',
                    'ServiceLines': ['AUDIT'],
                },
            ],
        },
    }
)
practice_session = MockAPISession(
    {
        (
            'https://api.practicecs.mock/v1/clients/validate',
            (('client_ids', 'C-1001,C-2040'),),
        ): {'valid_ids': ['C-1001', 'C-2040']},
        (
            'https://api.practicecs.mock/v1/service-lines',
            (('client_ids', 'C-1001,C-2040'),),
        ): {
            'service_lines': [
                {
                    'ClientID': 'C-1001',
                    'ServiceLineCode': 'CONSULT',
                    'Description': 'Consulting Services',
                    'Rate': 250,
                },
                {
                    'ClientID': 'C-1001',
                    'ServiceLineCode': 'TAX',
                    'Description': 'Tax Advisory',
                    'Rate': 180,
                },
                {
                    'ClientID': 'C-2040',
                    'ServiceLineCode': 'AUDIT',
                    'Description': 'Audit and Assurance',
                    'Rate': 310,
                },
            ]
        },
    }
)
deltek_api = MockDeltekVantagepointAPI(deltek_session)
practice_api = MockPracticeCSAPI(practice_session)
client_metadata = deltek_api.fetch_client_metadata('2025')
validated_ids = practice_api.validate_client_ids([c['ClientID'] for c in client_metadata])
service_line_catalog = practice_api.fetch_service_lines(validated_ids)
service_lines = [
    {'ServiceLineCode': code, **details}
    for code, details in service_line_catalog.items()
]
print('Client metadata records:', client_metadata)
print('Service line records:', service_lines)


## 2. Create sample engagement data
We'll stage demo metadata, service line rates, and a template document inside `./data/Finance/Engagements` for reporting period **202501**.

In [ ]:
from pathlib import Path
import json
from amplify_automations.core.io_utils import write_excel

PERIOD = '202501'
SUPPORT_DIR = Path('./data/Finance/Engagements')
SUPPORT_DIR.mkdir(parents=True, exist_ok=True)

metadata_path = SUPPORT_DIR / f'client_metadata_{PERIOD}.json'
metadata_path.write_text(json.dumps(client_metadata, indent=2), encoding='utf-8')

write_excel(
    service_lines,
    (SUPPORT_DIR / 'service_lines.xlsx').as_posix(),
    headers=['ServiceLineCode', 'Description', 'Rate'],
)

template_path = SUPPORT_DIR / 'Engagement_Letter_Template.dotx'
template_path.write_text(
    (
        'Engagement Letter for {{ClientName}}\n'
        'Services:\n{{ServiceSummary}}\n'
        'Fiscal Year FY{{FiscalYear}}\nPrepared {{GeneratedOn}}\n'
    ),
    encoding='utf-8',
)

print('Client metadata →', metadata_path)
print('Service lines workbook →', SUPPORT_DIR / 'service_lines.xlsx')
print('Template path →', template_path)


## 3. Review the staged inputs
Inspect the JSON metadata and the Excel service line reference to understand the structure expected by the automation.

In [ ]:
import json
from amplify_automations.core.io_utils import read_excel

with open(metadata_path, encoding='utf-8') as f:
    print(json.dumps(json.load(f), indent=2))

service_table = read_excel((SUPPORT_DIR / 'service_lines.xlsx').as_posix())
service_table

## 4. Configure and run the step
We provide folder mappings, parameter placeholders, and then execute the `ClientEngagementLetterDraft` step to generate draft letters.

In [ ]:
from amplify_automations.plugins.client_engagement_letter_draft import ClientEngagementLetterDraft
from amplify_automations.core.contracts import StepIO

cfg = {
    'params': {
        'client_metadata': '{support}/client_metadata_{period}.json',
        'service_lines': '{support}/service_lines.xlsx',
        'template_path': '{support}/Engagement_Letter_Template.dotx',
        'output_folder': '{support}/Drafts_{period}',
        'manifest_path': '{support}/Drafts_{period}/manifest.json',
        'notification_log': '{support}/Drafts_{period}/notifications.txt',
        'notification_recipients': ['teams://StaffAccountant'],
    }
}
folders = {'support': str(SUPPORT_DIR), 'root': './data/Finance'}
naming = {}

step = ClientEngagementLetterDraft(cfg, folders, naming, PERIOD)
io_plan: StepIO = step.plan_io()
result = step.run(io_plan)
print('Success:', result.ok)
print('Messages:', result.messages)
result.metrics

## 5. Inspect generated artifacts
The step outputs letters (as `.docx` text files for the tutorial), a JSON manifest, and a notification log summarising who to alert.

In [ ]:
from pathlib import Path
letters_dir = Path(io_plan.outputs['letters_dir'])
manifest_path = Path(io_plan.outputs['manifest'])
notification_path = Path(io_plan.outputs['notification_log'])

print('Letters directory:', letters_dir)
print('Generated files:', [p.name for p in letters_dir.glob('*.docx')])

print('Manifest preview:')
print(manifest_path.read_text(encoding='utf-8'))

print('Notification log:')
print(notification_path.read_text(encoding='utf-8'))


## 6. Next steps
- Replace the sample metadata export with your CRM/ERP client roster.
- Expand `service_lines.xlsx` to include billing terms, partners, or delivery details.
- Drop a prior year folder into the config (`prior_letters_folder`) to roll forward letters.
- Integrate the step into a full finance pipeline or schedule it inside your orchestration tooling.